In [67]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from sympy import symbols, lambdify
from sympy import symbols, hermite
from torch.distributions import Normal
from numpy.polynomial.hermite import Hermite, herm2poly

In [68]:
# ------------------------
# Snippet 0: Device Setup
# ------------------------
# Use Apple M1’s GPU via MPS if available, else CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}\n")

Using device: mps



In [69]:
# ─────────────────────────────────────────────────────────────────────────────
# 1) DATA‐GEN HELPERS
# ─────────────────────────────────────────────────────────────────────────────

x_sym = symbols('x')

def sample_hermite_function(p, q, coeff_variance=1.0):
    """
    Returns a NumPy‐vectorized function f(x) = sum_{i=p}^q c_i H_i(x),
    with c_i ~ N(0, coeff_variance).
    """
    expr = 0
    for i in range(p, q+1):
        c_i = np.random.normal(scale=np.sqrt(coeff_variance))
        expr += c_i * hermite(i, x_sym)
    return lambdify(x_sym, expr, 'numpy')

In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BayesianTransformerRegressor(nn.Module):
    def __init__(
        self,
        input_dim=2,
        d_model=100,
        nhead=1,
        dim_feedforward=100,
        num_encoder_layers=1,
        p_dropout=0.1
    ):
        super().__init__()
        # ————————————
        # 1) Embed (x,y) → d_model + nonlinearity
        # ————————————
        self.embed = nn.Sequential(
            nn.Linear(input_dim, d_model),
            nn.ReLU(),
            nn.Dropout(p_dropout),

            nn.Linear(d_model, d_model),
            nn.Dropout(p_dropout)

        )

        # —————————————————————
        # 2) Transformer encoder stack
        # —————————————————————
        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=p_dropout,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_encoder_layers)

        # ———————————————————————
        # 3) Deep output head (2 extra layers)
        # ———————————————————————
        self.out_mlp = nn.Sequential(

            nn.Linear(d_model, 2),

        )

    def forward(self, seq):
        """
        seq: (batch, seq_len, 2) tensor of (x, masked_y)
        returns: (mean, std) each of shape (batch, seq_len)
        """
        # 1) embed + nonlinearity
        h = self.embed(seq)                # → (B, L, d_model)

        # 2) build causal mask so j > i positions are masked out
        L = h.size(1)
        mask = torch.triu(torch.full((L, L), float('-inf'), device=h.device), diagonal=1)

        # 3) transformer with causal attention
        h = self.encoder(h, mask=mask)     # → (B, L, d_model)

        # 4) deep MLP head to [mean, log_std]
        out = self.out_mlp(h)              # → (B, L, 2)
        mean    = out[..., 0]              # → (B, L)
        log_std = out[..., 1]
        std     = torch.exp(log_std)       # ensure positivity

        return mean, std

# instantiate and move to GPU/CPU
model = BayesianTransformerRegressor(
    input_dim=2,
    d_model=64,             # you can bump this up too
    nhead=1,
    dim_feedforward=64,
    num_encoder_layers=1,
    p_dropout=0.01
).to(device)

In [ ]:

# 1: Generate M training tasks, each length N, mask last y

#Give N_range
N_range = np.linspace(3,15,5, dtype=int)
p, q = 0, 3 
num_function = 10
# Generate M tasks of length N
for N in N_range:

    for i in range(1,num_function):
        M = 50
        tasks_x = np.random.normal(0, 1, size=(M, N))
        tasks_x.sort(axis=1)
        f = sample_hermite_function(p, q, coeff_variance = 1)
        tasks_y = f(tasks_x)
        tasks_y_masked = tasks_y.copy()
        tasks_y_masked[:, -1] = 0.0
        data = np.stack([tasks_x, tasks_y_masked], axis=2)  # (M, N, 2)

        # Move to torch tensors
        data_t    = torch.tensor(data, dtype=torch.float32).to(device)      # (M, N, 2)
        true_last = torch.tensor(tasks_y[:, -1], dtype=torch.float32).to(device)  # (M,)



        # 4️⃣ Training loop with negative log-likelihood
        # ---------------------------------------
        # Snippet 3: Training Loop with MC-Dropout
        # ---------------------------------------
        # Move data to device

        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        epochs    = 10000

        for ep in range(1, epochs+1):
            model.train()
            optimizer.zero_grad()

            mean_all, std_all = model(data_t)       # (M, N), (M, N)
            mean_last = mean_all[:, -1]             # (M,)
            std_last  = std_all[:, -1]              # (M,)

            dist = Normal(mean_last, std_last)
            log_p = dist.log_prob(true_last)        # (M,)
            loss  = -log_p.mean()                   # negative log-likelihood

            loss.backward()
            optimizer.step()

            if ep % 100 == 0:
                print(f"Epoch {ep}/{epochs} — NLL: {loss.item():.4f}")
                print(f"Mean: {mean_last.mean():.4f} ± {std_last.mean():.4f}")
                print(f"True last y: {true_last.mean():.4f} ± {true_last.std():.4f}")
                #Print which N_range value is being trained
                print(f"Training on N={N}...\n")
                print(f"Training on function {i}...\n")

        print('Training Complete')

        # 1e) Visualize task #0 with scatter
        plt.figure(figsize=(6,4))
        # Plot all masked training points as circles
        plt.scatter(
            data[0, :, 0],        # x-coordinates
            data[0, :, 1],        # masked y-values
            marker='o',
            color='tab:blue',
            label='masked y'
        )
        # Highlight the true last yₙ as a larger square
        plt.scatter(
            tasks_x[0, -1],       # xₙ
            tasks_y[0, -1],       # true yₙ
            marker='s',
            s=100,
            color='tab:red',
            label='true yₙ'
        )
        plt.title("Example Task (idx=0): Training Sequence with Masked Last Value")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()
        plt.show()


Epoch 100/10000 — NLL: 3.4025
Mean: -0.1479 ± 6.2098
True last y: -0.3638 ± 8.4985
Training on N=3...

Training on function 1...

Epoch 200/10000 — NLL: 3.1446
Mean: -0.1836 ± 5.7083
True last y: -0.3638 ± 8.4985
Training on N=3...

Training on function 1...

Epoch 300/10000 — NLL: 3.0208
Mean: -0.1003 ± 5.3772
True last y: -0.3638 ± 8.4985
Training on N=3...

Training on function 1...

Epoch 400/10000 — NLL: 2.9447
Mean: 0.2459 ± 5.1093
True last y: -0.3638 ± 8.4985
Training on N=3...

Training on function 1...

Epoch 500/10000 — NLL: 2.7635
Mean: 0.8665 ± 4.7276
True last y: -0.3638 ± 8.4985
Training on N=3...

Training on function 1...

Epoch 600/10000 — NLL: 2.4798
Mean: 1.6024 ± 3.9324
True last y: -0.3638 ± 8.4985
Training on N=3...

Training on function 1...

Epoch 700/10000 — NLL: 1.8204
Mean: 2.1859 ± 2.7759
True last y: -0.3638 ± 8.4985
Training on N=3...

Training on function 1...

Epoch 800/10000 — NLL: 1.3155
Mean: 2.5177 ± 3.0084
True last y: -0.3638 ± 8.4985
Training on 

In [ ]:
for i in range(1, num_function):
    # 5️⃣ MC-Dropout Inference on test_num random test tasks
    test_num, T = 100, 100
    f = sample_hermite_function(p, q, coeff_variance=1.0)
    # Create new random tasks
    tx = np.random.normal(-1, 1, size=(test_num, 10))
    tx.sort(axis=1)
    ty = f(tx)
    ty_mask = ty.copy()
    ty_mask[:, -1] = 0.0
    data_rand   = np.stack([tx, ty_mask], axis=2)
    data_rand_t = torch.tensor(data_rand, dtype=torch.float32).to(device)

    model.train()  # keep dropout active
    all_samps = []

    for _ in range(T):
        with torch.no_grad():
            m_r, s_r = model(data_rand_t)
            # sample y_n ~ Normal(mean, std)
            samp = m_r[:, -1] + s_r[:, -1] * torch.randn(test_num, device=device)
            all_samps.append(samp.cpu().numpy())

    all_samps = np.stack(all_samps, axis=0)      # (T, test_num)
    mean_pred = all_samps.mean(axis=0)           # (test_num,)
    std_pred  = all_samps.std(axis=0)

    # Plot predictions ±2σ vs. true y_n
    plt.figure(figsize=(8,4))
    plt.errorbar(
        tx[:, -1], mean_pred,
        yerr=2*std_pred,
        fmt='o', ecolor='lightgray', capsize=3,
        label='Predicted ±2σ'
    )
    plt.scatter(
        tx[:, -1], ty[:, -1],
        color='red', marker='x', s=50,
        label='True yₙ'
    )
    plt.title("MC-Dropout on 100 Random Test Tasks")
    plt.xlabel("xₙ")
    plt.ylabel("yₙ")
    plt.legend()
    plt.show()


